In [ ]:
mol1_features = np.random.rand(n_datapoints, n_features1)
mol2_features = np.random.rand(n_datapoints, n_features2)
x_ds = np.hstack([mol1_features, mol2_features])
mol1s = [List of control molecules]
mol2s = [List of target molecules]
ys = np.random.rand(n_datapoints, n_tasks)
datapoints1 = [MoleculeDatapoint(mol, y, x_d=x_d) for mol, y, x_d in zip(mol1s, ys, x_ds)]
datapoints2 = [MoleculeDatapoint(mol) for mol in mol2s]
dataset1 = MoleculeDataset(datapoints1)
dataset2 = MoleculeDataset(datapoints2)
multi_dataset = MulticomponentDataset(datasets=[dataset1, dataset2])
dataloader = chemprop.data.build_dataloader(multi_dataset)

In [1]:
import chemprop
import rdkit
import torch

mol = rdkit.Chem.MolFromSmiles("CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C")  # Penicillin
dp = chemprop.data.MoleculeDatapoint(mol)
ds = chemprop.data.MoleculeDataset([dp])

In [2]:
%timeit list(mol.GetBonds())
n_atoms = mol.GetNumAtoms()
%timeit [mol.GetBondBetweenAtoms(u, v) for u in range(n_atoms) for v in range(u+1, n_atoms)]
bond = mol.GetBondBetweenAtoms(0, 1)
%timeit u, v = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()

22.5 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
66.8 µs ± 582 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
342 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [2]:
# Original
%timeit ds[0]
ds.cache = True
%timeit ds[0]
ds[0][0].edge_index

340 µs ± 32.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
455 ns ± 6.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


array([[ 0,  1,  1,  2,  1,  5,  1, 22,  2,  3,  2, 19,  3,  4,  3,  7,
         4,  5,  4,  6,  6,  7,  6,  9,  7,  8,  9, 10, 10, 11, 10, 12,
        12, 13, 13, 14, 13, 18, 14, 15, 15, 16, 16, 17, 17, 18, 19, 20,
        19, 21],
       [ 1,  0,  2,  1,  5,  1, 22,  1,  3,  2, 19,  2,  4,  3,  7,  3,
         5,  4,  6,  4,  7,  6,  9,  6,  8,  7, 10,  9, 11, 10, 12, 10,
        13, 12, 14, 13, 18, 13, 15, 14, 16, 15, 17, 16, 18, 17, 20, 19,
        21, 19]])

In [2]:
# GetBonds
%timeit ds[0]
ds.cache = True
%timeit ds[0]
ds[0][0].edge_index

263 µs ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
439 ns ± 8.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


array([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  4,  6,  6,  7,  7,  8,
         6,  9,  9, 10, 10, 11, 10, 12, 12, 13, 13, 14, 14, 15, 15, 16,
        16, 17, 17, 18,  2, 19, 19, 20, 19, 21,  1, 22,  5,  1,  7,  3,
        18, 13],
       [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  4,  7,  6,  8,  7,
         9,  6, 10,  9, 11, 10, 12, 10, 13, 12, 14, 13, 15, 14, 16, 15,
        17, 16, 18, 17, 19,  2, 20, 19, 21, 19, 22,  1,  1,  5,  3,  7,
        13, 18]])

In [2]:
# List comprehension
%timeit ds[0]
ds.cache = True
%timeit ds[0]
ds[0][0].edge_index

287 µs ± 6.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
439 ns ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


array([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  4,  6,  6,  7,  7,  8,
         6,  9,  9, 10, 10, 11, 10, 12, 12, 13, 13, 14, 14, 15, 15, 16,
        16, 17, 17, 18,  2, 19, 19, 20, 19, 21,  1, 22,  5,  1,  7,  3,
        18, 13],
       [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  4,  7,  6,  8,  7,
         9,  6, 10,  9, 11, 10, 12, 10, 13, 12, 14, 13, 15, 14, 16, 15,
        17, 16, 18, 17, 19,  2, 20, 19, 21, 19, 22,  1,  1,  5,  3,  7,
        13, 18]])

In [2]:
mol = rdkit.Chem.MolFromSmiles("")
dp = chemprop.data.MoleculeDatapoint(mol)
ds = chemprop.data.MoleculeDataset([dp])

In [3]:
ds[0]

Datum(mg=MolGraph(V=array([], shape=(0, 72), dtype=float64), E=array([], shape=(0, 14), dtype=float64), edge_index=array([], shape=(2, 0), dtype=int64), rev_edge_index=array([], dtype=int64)), V_d=None, x_d=None, y=nan, weight=1.0, lt_mask=None, gt_mask=None)

In [4]:
mp = chemprop.nn.BondMessagePassing()

In [5]:
dl = chemprop.data.build_dataloader(ds)

In [6]:
batch = next(iter(dl))

In [10]:
mg, *_ = batch

In [11]:
mp(mg, None)

tensor([], size=(0, 300), grad_fn=<ReluBackward0>)

In [2]:
mol = rdkit.Chem.MolFromSmiles("")
dp = chemprop.data.MoleculeDatapoint(mol)
ds = chemprop.data.MoleculeDataset([dp])

In [1]:
import chemprop
import rdkit
import torch

mol = rdkit.Chem.MolFromSmiles("CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C")  # Penicillin
dp = chemprop.data.MoleculeDatapoint(mol)
ds = chemprop.data.MoleculeDataset([dp])
dl = chemprop.data.build_dataloader(ds, batch_size=1)
batch = next(iter(dl))
mg, *_ = batch

In [2]:
checkpoint_path = "tests/data/example_model_v2_regression_mol.ckpt"
mpnn = chemprop.models.MPNN.load_from_checkpoint(checkpoint_path)
mpnn

/home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.2.2, which is newer than your current Lightning version: v2.2.1


MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=300, bias=False)
    (W_h): Linear(in_features=300, out_features=300, bias=False)
    (W_o): Linear(in_features=372, out_features=300, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): GraphTransform(
      (V_transform): Identity()
      (E_transform): Identity()
    )
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=300, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=300, out_features=1, bias=True)
      )
    )
    (criterion): MSELoss(task_weights=[[1.0]])
    (output_transform): UnscaleTransform()
  )
  (X_d_transform): Identity()
)

In [13]:
mpnn.message_passing(mg, None)

tensor([[0.0146, 0.0000, 0.0711,  ..., 0.1050, 0.1969, 0.0000],
        [0.0836, 0.0000, 0.0000,  ..., 0.2042, 0.3485, 0.2322],
        [0.0000, 0.0000, 0.1735,  ..., 0.1548, 0.2860, 0.0000],
        ...,
        [0.0000, 0.0398, 0.2541,  ..., 0.0511, 0.0206, 0.0000],
        [0.0000, 0.0000, 0.0603,  ..., 0.0074, 0.1134, 0.0000],
        [0.0146, 0.0000, 0.0711,  ..., 0.1050, 0.1969, 0.0000]],
       grad_fn=<ReluBackward0>)

In [3]:
mpnn.message_passing(mg, None)

tensor([[0.0146, 0.0000, 0.0711,  ..., 0.1050, 0.1969, 0.0000],
        [0.0836, 0.0000, 0.0000,  ..., 0.2042, 0.3485, 0.2322],
        [0.0000, 0.0000, 0.1735,  ..., 0.1548, 0.2860, 0.0000],
        ...,
        [0.0000, 0.0398, 0.2541,  ..., 0.0511, 0.0206, 0.0000],
        [0.0000, 0.0000, 0.0603,  ..., 0.0074, 0.1134, 0.0000],
        [0.0146, 0.0000, 0.0711,  ..., 0.1050, 0.1969, 0.0000]],
       grad_fn=<ReluBackward0>)

In [3]:
import torch
import torchmetrics

In [16]:
a = torch.randn(10, 5)
b = torch.randn(10, 5)

In [6]:
m = torchmetrics.MeanSquaredError()

In [32]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = torch.nn.functional.mse_loss(input, target, reduction="none")

In [33]:
output

tensor([[1.8324e-01, 2.7586e+00, 3.8411e+00, 2.0319e+00, 4.2811e+00],
        [1.0199e+00, 1.0741e+00, 6.2624e-02, 2.0293e-04, 6.3696e-02],
        [1.4223e+00, 3.7240e-03, 1.6341e+00, 3.9651e+00, 1.1566e-01]],
       grad_fn=<MseLossBackward0>)

In [41]:
a = torch.load(
    "/home/knathan/chemprop/chemprop_training/regression/2024-08-15T16-27-37/model_0/checkpoints/best-3-metric=0.05.ckpt"
)

In [39]:
a["state_dict"]["metrics.0.task_weights"] = torch.tensor([1.0])

In [42]:
a["state_dict"]

OrderedDict([('message_passing.W_i.weight',
              tensor([[ 0.0706,  0.0169, -0.0455,  ..., -0.0773,  0.0912,  0.0133],
                      [-0.0699, -0.0489, -0.0614,  ...,  0.0055, -0.0094,  0.0202],
                      [-0.0587, -0.0157,  0.0461,  ...,  0.0138, -0.0762,  0.1000],
                      ...,
                      [-0.0399,  0.0559,  0.0477,  ..., -0.0595,  0.0645, -0.0370],
                      [ 0.0699, -0.0749,  0.1031,  ...,  0.0249,  0.1032, -0.0333],
                      [-0.0739,  0.0413, -0.0260,  ..., -0.0189,  0.0846,  0.0117]])),
             ('message_passing.W_h.weight',
              tensor([[ 0.0551,  0.0161, -0.0060,  ..., -0.0489, -0.0355, -0.0217],
                      [ 0.0240, -0.0067,  0.0056,  ..., -0.0368,  0.0477, -0.0200],
                      [ 0.0155, -0.0106, -0.0087,  ..., -0.0260,  0.0372,  0.0429],
                      ...,
                      [ 0.0057, -0.0094,  0.0189,  ..., -0.0023,  0.0232, -0.0370],
               

In [46]:
a["hyper_parameters"]["predictor"]

"activation":       RELU
"cls":              <class 'chemprop.nn.predictors.RegressionFFN'>
"criterion":        None
"dropout":          0.0
"hidden_dim":       300
"input_dim":        300
"n_layers":         1
"n_tasks":          1
"output_transform": UnscaleTransform()
"task_weights":     None
"threshold":        None